In [1]:
####################################
## A. Librerias
####################################
import os
import time
import json
import pandas as pd

from utils.paths import CHROMEDRIVER_PATH

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def get_chrome_driver(chromedriver_path=None, headless=False):
    # Configurar las opciones de Selenium
    options = webdriver.ChromeOptions()

    options.add_argument("--no-sandbox")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--ignore-certificate-errors")
    
    if headless:
        options.add_argument("--headless=new")

    if chromedriver_path is not None:
        print("Usando el chromedriver_path: {}".format(chromedriver_path))
        service = Service(executable_path=chromedriver_path)
        driver = webdriver.Chrome(service=service, options=options)
    else:
        print("Usando el chromedriver_path por defecto")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    return driver

ESTAMOS EN --->  darwin
PROJECT_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper
CHROME_DRIVER_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver



In [2]:
########################
## C. Ejecución
########################

# 1. Seleccionar modo
HEADLESS = False
PRINT_VIEW = False

# 2. Eliminar todos los archivos de la carpeta
try:
    files = get_files(os.path.join(os.getcwd(),'outputs','jsons','renacyt_profile'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

# 3. Leer los jsons y guardarlos en un dataframe
df_renacyt= pd.read_parquet(os.path.join(os.getcwd(),'outputs','parquets','renacyt.parquet'))
df_renacyt = df_renacyt[df_renacyt['ctiVitae'].notnull()]
df_renacyt = df_renacyt[df_renacyt['ctiVitae'] != 'nan']
df_renacyt = df_renacyt[df_renacyt['ctiVitae'] != 'None']
df_renacyt = df_renacyt[df_renacyt['ctiVitae'] != '']
df_renacyt['ctiVitae'] = df_renacyt['ctiVitae'].astype(str)
df_renacyt.drop_duplicates(subset=['ctiVitae'], inplace=True)
df_renacyt = df_renacyt.astype(str)

# 4. Crear todas las URLs
df_renacyt = df_renacyt[['orcid','ctiVitae']]
df_renacyt['url'] = 'https://ctivitae.concytec.gob.pe/buscador-ui/#/ficha/ficha-renacyt?idInvestigador=' + df_renacyt['ctiVitae']
#df_renacyt = df_renacyt[['orcid','url']]

df_renacyt = df_renacyt.head(10)
df_renacyt

Se eliminaron los archivos de la carpeta


,orcid,ctiVitae,url
738,0000-0003-1583-7113,10,https://ctivitae.concytec.gob.pe/buscador-ui/#...
5797,None,13,https://ctivitae.concytec.gob.pe/buscador-ui/#...
5749,0000-0001-9740-5490,16,https://ctivitae.concytec.gob.pe/buscador-ui/#...
3308,0000-0002-8301-5536,17,https://ctivitae.concytec.gob.pe/buscador-ui/#...
6842,0000-0002-8207-9714,19,https://ctivitae.concytec.gob.pe/buscador-ui/#...
4765,0000-0002-9695-5746,21,https://ctivitae.concytec.gob.pe/buscador-ui/#...
5968,0000-0003-3338-2503,25,https://ctivitae.concytec.gob.pe/buscador-ui/#...
5004,0000-0003-2259-5327,26,https://ctivitae.concytec.gob.pe/buscador-ui/#...
7204,0000-0002-8069-2101,34,https://ctivitae.concytec.gob.pe/buscador-ui/#...
5917,0000-0001-5560-9241,43,https://ctivitae.concytec.gob.pe/buscador-ui/#...


In [3]:
######################
## D. Leer los jsons
######################

# Iniciar el driver
driver = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, headless=HEADLESS)

# Leer los jsons
url = 'https://ctivitae.concytec.gob.pe/buscador-ui/#/ficha/ficha-renacyt?idInvestigador=122873'
driver.get(url)
print(url)

# Esperar a que la tabla específica esté presente en el DOM
wait = WebDriverWait(driver, 10)
table_locator = (By.CSS_SELECTOR, 'table.table.table-sm')
table = wait.until(EC.presence_of_element_located(table_locator))

Usando el chromedriver_path: /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver
https://ctivitae.concytec.gob.pe/buscador-ui/#/ficha/ficha-renacyt?idInvestigador=10


In [4]:
# Obtener el contenido HTML después de que la página se ha cargado completamente
html = driver.page_source
html

'<html lang="en"><head><base href="/buscador-ui/">\n  <!--<meta charset="utf-8">\n  <title>Registro de Investigadores - RENACYT</title>\n  <base href="/">\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  <link rel="icon" type="image/x-icon" href="favicon.ico">\n  <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.8.2/css/all.css" integrity="sha384-oS3vJWv+0UjzBfQzYUhtDYW+Pj2yciDJxpsK1OYPAYjqT085Qq/1cq5FLXAZQ7Ay" crossorigin="anonymous">-->\n\n<link rel="stylesheet" href="styles.287a7c37de19c3cedb81.css"><style>.swal2-popup.swal2-toast{-webkit-box-orient:horizontal;-webkit-box-direction:normal;flex-direction:row;-webkit-box-align:center;align-items:center;width:auto;padding:.625em;overflow-y:hidden;background:#fff;box-shadow:0 0 .625em #d9d9d9}.swal2-popup.swal2-toast .swal2-header{-webkit-box-orient:horizontal;-webkit-box-direction:normal;flex-direction:row}.swal2-popup.swal2-toast .swal2-title{-webkit-box-flex:1;flex-grow:1;-webkit-box-pack:

In [14]:
# Analizar el HTML con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Encuentra la tabla por su clase
tablas = soup.find_all('table')

# Inicializa una lista para almacenar DataFrames de cada tabla
dataframes = []

# Itera sobre las tablas y extrae los datos
for i, tabla in enumerate(tablas):
    # Inicializa listas para almacenar filas y columnas de cada tabla
    filas = []
    columnas = []

    # Encuentra las filas en el cuerpo de la tabla
    filas_html = tabla.find('tbody').find_all('tr') if tabla.find('tbody') else tabla.find_all('tr')
    
    # Itera sobre las filas y extrae los datos de las columnas
    for fila_html in filas_html:
        columnas_html = fila_html.find_all(['td', 'th'])  # Incluyendo 'th' para encabezados de columna
        fila = [columna.text.strip() for columna in columnas_html]
        columnas.append(fila)

    # Crea un DataFrame con los datos de la tabla actual
    df = pd.DataFrame(columnas)

    # Añade el DataFrame a la lista
    dataframes.append(df)


In [32]:
dataframes[12]

IndexError: list index out of range

In [6]:
driver.quit()